In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
attrition_df = pd.read_csv('Take home assessment Doc_ Sample_Employee_Termination_Data.csv',
                           parse_dates=['recorddate_key', 'orighiredate_key', 'terminationdate_key'])

In [3]:
attrition_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49653 entries, 0 to 49652
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   EmployeeID           49653 non-null  int64         
 1   recorddate_key       49653 non-null  datetime64[ns]
 2   birthdate_key        49653 non-null  object        
 3   orighiredate_key     49653 non-null  datetime64[ns]
 4   terminationdate_key  49653 non-null  datetime64[ns]
 5   age                  49653 non-null  int64         
 6   length_of_service    49653 non-null  int64         
 7   city_name            49653 non-null  object        
 8   department_name      49653 non-null  object        
 9   job_title            49653 non-null  object        
 10  store_name           49653 non-null  int64         
 11  gender_short         49653 non-null  object        
 12  gender_full          49653 non-null  object        
 13  termreason_desc      49653 non-

In [4]:
attrition_df.head()

,EmployeeID,recorddate_key,birthdate_key,orighiredate_key,terminationdate_key,age,length_of_service,city_name,department_name,job_title,store_name,gender_short,gender_full,termreason_desc,termtype_desc,STATUS_YEAR,STATUS,BUSINESS_UNIT
0,1318,2006-12-31,1/3/54,1989-08-28,2000-01-01,52,17,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2006,ACTIVE,HEADOFFICE
1,1318,2007-12-31,1/3/54,1989-08-28,2000-01-01,53,18,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2007,ACTIVE,HEADOFFICE
2,1318,2008-12-31,1/3/54,1989-08-28,2000-01-01,54,19,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2008,ACTIVE,HEADOFFICE
3,1318,2009-12-31,1/3/54,1989-08-28,2000-01-01,55,20,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2009,ACTIVE,HEADOFFICE
4,1318,2010-12-31,1/3/54,1989-08-28,2000-01-01,56,21,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2010,ACTIVE,HEADOFFICE


In [5]:
# function to fix birthdate_key series
def fix_date(x: pd.Series) -> datetime.date:
    month = int(x.split('/')[0])
    day = int(x.split('/')[1])
    year = int('19' + x.split('/')[2])
    return datetime.date(year, month, day)

In [6]:
attrition_df['birthdate'] = attrition_df['birthdate_key'].apply(fix_date)
attrition_df['birthdate']

0        1954-01-03
1        1954-01-03
2        1954-01-03
3        1954-01-03
4        1954-01-03
            ...    
49648    1994-05-28
49649    1994-06-13
49650    1994-07-18
49651    1994-09-02
49652    1994-11-28
Name: birthdate, Length: 49653, dtype: object

In [7]:
# Dropping unneeded column
attrition_df = attrition_df.drop(columns=['birthdate_key'])

In [8]:
# Grouping columns by EmployeeID to get latest relevant row
att_group = attrition_df.groupby('EmployeeID')

In [9]:
attrition_df_cleaned = att_group.apply(lambda x: x[x['length_of_service'] == x['length_of_service'].max()])

In [10]:
attrition_df_cleaned['job_title_general'] = np.nan
attrition_df_cleaned.loc[attrition_df_cleaned['job_title'].str.contains('VP'), 'job_title_general'] = 'VP'
attrition_df_cleaned.loc[attrition_df_cleaned['job_title'].str.contains('Manager'), 'job_title_general'] = 'Manager'
attrition_df_cleaned.loc[attrition_df_cleaned['job_title'].str.contains('Director'), 'job_title_general'] = 'Director'
attrition_df_cleaned.loc[attrition_df_cleaned['job_title'].str.contains('Analyst'), 'job_title_general'] = 'Analyst'

In [11]:
# Confirming change
attrition_df_cleaned.loc[attrition_df_cleaned['job_title_general'].isin(['VP', 'Manager', 'Director', 'Analyst'])]

,,EmployeeID,recorddate_key,orighiredate_key,terminationdate_key,age,length_of_service,city_name,department_name,job_title,store_name,gender_short,gender_full,termreason_desc,termtype_desc,STATUS_YEAR,STATUS,BUSINESS_UNIT,birthdate,job_title_general
EmployeeID,,,,,,,,,,,,,,,,,,,,
1319,19,1319,2015-12-31,1989-08-28,2000-01-01,58,26,Vancouver,Executive,VP Stores,35,F,Female,Not Applicable,Not Applicable,2015,ACTIVE,HEADOFFICE,1957-01-03,VP
1321,39,1321,2015-12-31,1989-08-28,2000-01-01,56,26,Vancouver,Executive,VP Human Resources,35,M,Male,Not Applicable,Not Applicable,2015,ACTIVE,HEADOFFICE,1959-01-02,VP
1322,49,1322,2015-12-31,1989-08-31,2000-01-01,57,26,Vancouver,Executive,VP Finance,35,M,Male,Not Applicable,Not Applicable,2015,ACTIVE,HEADOFFICE,1958-01-09,VP
1323,59,1323,2015-12-31,1989-08-31,2000-01-01,53,26,Vancouver,Executive,"Exec Assistant, VP Stores",35,M,Male,Not Applicable,Not Applicable,2015,ACTIVE,HEADOFFICE,1962-01-09,VP
1329,89,1329,2015-12-31,1989-09-08,2000-01-01,48,26,Terrace,Store Management,Store Manager,32,F,Female,Not Applicable,Not Applicable,2015,ACTIVE,STORES,1967-01-23,Manager
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3850,48758,3850,2014-12-01,1996-10-21,2014-12-30,52,18,Fort Nelson,Customer Service,Customer Service Manager,11,M,Male,Layoff,Involuntary,2014,TERMINATED,STORES,1962-07-18,Manager
3970,48781,3970,2014-12-01,1997-02-21,2014-12-30,51,17,New Westminister,Processed Foods,Processed Foods Manager,20,M,Male,Layoff,Involuntary,2014,TERMINATED,STORES,1963-03-07,Manager
3994,48787,3994,2014-12-01,1997-03-16,2014-12-30,51,17,Grand Forks,Bakery,Bakery Manager,13,F,Female,Layoff,Involuntary,2014,TERMINATED,STORES,1963-04-19,Manager


In [12]:
# Reordering columns
attrition_df_final = attrition_df_cleaned[['EmployeeID',
 'recorddate_key',
 'orighiredate_key',
 'birthdate',
 'terminationdate_key',
 'age',
 'length_of_service',
 'city_name',
 'department_name',
 'job_title_general',
 'store_name',
 'gender_short',
 'gender_full',
 'termreason_desc',
 'termtype_desc',
 'STATUS_YEAR',
 'STATUS',
 'BUSINESS_UNIT',
]]

In [14]:
# exporting for visualization
attrition_df_final.to_csv('Sample_Employee_Termination_Data_cleaned.csv', index=False)